# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [25]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/Examples"

### Modeling Packages

In [26]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [27]:
using Dates
using DataFrames

### Optimization packages

In [28]:
using JuMP
using Cbc # solver
Cbc_optimizer = JuMP.with_optimizer(Cbc.Optimizer, logLevel = 1, ratioGap = 0.5)

OptimizerFactory(Cbc.Optimizer, (), Base.Iterators.Pairs{Symbol,Real,Tuple{Symbol,Symbol},NamedTuple{(:logLevel, :ratioGap),Tuple{Int64,Float64}}}(:logLevel => 1,:ratioGap => 0.5))

### Data
There is a meaningless test dataset assembled in the
[make_hydropower_data.jl](../../script/PowerSimulations_examples/make_hydro_data.jl) script.

In [29]:
include(joinpath(pkgpath, "script/PowerSimulations_examples/make_hydro_data.jl"))

┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.07
│   valid_range = voltagelimits
│   valid_info.ist_struct = Bus(6, "Bus 6", PowerSystems.PV, -0.24818581963359368, 1.07, (min = 0.94, max = 1.06), 13.8, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("249e32bf-b13d-49e8-84b8-8e7de68503de"), nothing))
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Tjbyn/src/validation.jl:202
┌ Warning: Invalid range
│   valid_info.struct_name = Bus
│   field_name = voltage
│   field_value = 1.062
│   valid_range = voltagelimits
│   valid_info.ist_struct = Bus(7, "Bus 7", PowerSystems.PQ, -0.23335052099164186, 1.062, (min = 0.94, max = 1.06), 13.8, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("e0b85710-b597-4984-8e85-ade7871de549"), nothing))
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/Tjbyn/src/validation.jl:202
┌ Warnin

## Two PowerSimulations features determine hydropower representation.
There are two prinicpal ways that we can customize hydropower representation in
PowerSimulations. First, we can play with the formulation applied to hydropower generators
using the `DeviceModel`. We can also adjust how simulaitons are configured to represent
different decison making processes and the information flow between those processes.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [30]:
TypeTree(PSY.HydroGen)

HydroGen
├──HydroFix
└──HydroDispatch

And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [31]:
TypeTree(PSI.AbstractHydroFormulation)

PowerSimulations.AbstractHydroFormulation
├──PowerSimulations.AbstractHydroDispatchFormulation
│  ├──HydroDispatchReservoirFlow
│  ├──HydroDispatchRunOfRiver
│  └──HydroDispatchReservoirStorage
├──HydroFixed
└──PowerSimulations.AbstractHydroUnitCommitment
   ├──HydroCommitmentRunOfRiver
   ├──HydroCommitmentReservoirFlow
   └──HydroCommitmentReservoirStorage

Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroDispatch` generators, and the
`HydroFixed` formulation to `HydroFix` generators.
 - The `HydroFixed` formulaton just acts
like a load subtractor, forcing the system to accept it's generation.
 - The `HydroDispatchRunOfRiver` formulation represents the the energy flowing out of
a reservoir. The model can choose to produce power with that energy or just let it spill by.

In [32]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :Hyd2 => DeviceModel(HydroFix, HydroFixed),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

┌ Info: The optimization model has no optimizer attached
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:14
┌ Info: Instantiating the JuMP model
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:16
┌ Info: Building PowerLoad with StaticPowerLoad formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building HydroDispatch with HydroDispatchRunOfRiver formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building HydroFix with HydroFixed formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building CopperPlatePowerModel network formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:391
┌ Info:


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Hyd2:
        device_type = HydroFix
        formulation = HydroFixed
  branches: 
  services: 


Now we can see the resulting JuMP model:

In [33]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

The first two constraints are the power balance constraints that require the generation
from the controllable `HydroDispatch` generators to be equal to the load (flat 10.0 for all time periods)
minus the generation from the `HydroFix` generators [1.97, 1.983, ...]. The 3rd and 4th
constraints limit the output of the `HydroDispatch` generator to the limit defined by the
`max_activepwoer` forecast. And the last 4 constraints are the lower and upper bounds of
the `HydroDispatch` operating range.

Next, let's apply the `HydroDispatchReservoirFlow` formulation to the `HydroDispatch` generators.

In [34]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirFlow),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

┌ Info: The optimization model has no optimizer attached
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:14
┌ Info: Instantiating the JuMP model
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:16
┌ Info: Building PowerLoad with StaticPowerLoad formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building HydroDispatch with HydroDispatchReservoirFlow formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building CopperPlatePowerModel network formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:391
┌ Info: Building Objective
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:401



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroDispatchReservoirFlow
  branches: 
  services: 


And, the resulting JuMP model:

In [35]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Finally, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroDispatch` generators.

In [36]:
devices = Dict{Symbol,DeviceModel}(
    :Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirStorage),
    :Load => DeviceModel(PowerLoad, StaticPowerLoad),
);

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

┌ Info: The optimization model has no optimizer attached
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:14
┌ Info: Instantiating the JuMP model
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:16
┌ Info: Building PowerLoad with StaticPowerLoad formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building HydroDispatch with HydroDispatchReservoirStorage formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Setting DeviceEnergy initial conditions for the status of all devices HydroDispatch based on system data
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/routines/make_initial_conditions.jl:151
┌ Info: Building CopperPlatePowerModel network formulation
└ @ PowerSimulations /Users/cbarrows/.julia/


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Load:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroDispatchReservoirStorage
  branches: 
  services: 


In [37]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

### Multi-Stage `SimulationSequence`
The purpsoe of a multi-stage simulaiton is to represent scheduling decisions consistently
with the time scales that govern different elements of power systems.

Multi-Day to Daily Simulation:

In the multi-day model, we'll use a really simple representation of all system devices
so that we can maintain computational tractability while getting an estimate of system
requirements/capabilities.

In [38]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatchNoMin),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroDispatch => DeviceModel(HydroDispatch, HydroDispatchReservoirStorage),
)
template_md = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

For the daily model, we can increase the modeling detail since we'll be solving shorter
problems.

In [39]:
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatchNoMin),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroDispatch => DeviceModel(HydroDispatch, HydroDispatchReservoirFlow),
)
template_da = OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

In [40]:
stages_definition = Dict(
    "MD" => Stage(GenericOpProblem, template_md, c_sys5_hy_wk, Cbc_optimizer),
    "DA" => Stage(GenericOpProblem, template_da, c_sys5_hy_uc, Cbc_optimizer),
)

Dict{String,Stage{GenericOpProblem}} with 2 entries:
  "DA" => Stage()…
  "MD" => Stage()…

Thsi builds the sequence and passes the the enregy dispatch schedule for the `HydroDispatch`
generatorfrom the "MD" stage to the "DA" stage in the form of an energy limit over the
synchronized periods.

In [41]:
sequence = SimulationSequence(
    order = Dict(1 => "MD", 2 => "DA"),
    intra_stage_chronologies = Dict(("MD" => "DA") => Synchronize(periods = 2)),
    horizons = Dict("MD" => 2, "DA" => 24),
    intervals = Dict("MD" => Hour(48), "DA" => Hour(24)),
    feed_forward = Dict(
        ("DA", :devices, :HydroDispatch) =>
                IntegralLimitFF(variable_from_stage = :P, affected_variables = [:P]),
    ),
    ini_cond_chronology = Dict("MD" => Consecutive(), "DA" => Consecutive()),
)

SimulationSequence()


In [42]:
file_path = tempdir()

sim = Simulation(
    name = "hydro",
    steps = 1,
    step_resolution = Hour(48),
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
    verbose = true,
)

build!(sim)

┌ Warning: Initial time not defined as an argument, it will be infered from the data.
│                Initial Simulation Time set to 2024-01-01T00:00:00
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/simulation.jl:197
┌ Info: Building Stage 2-DA
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/simulation.jl:243
┌ Info: Instantiating the JuMP model
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:16
┌ Info: Building HydroDispatch with HydroDispatchReservoirFlow formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building ThermalStandard with ThermalDispatchNoMin formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building PowerLoad with StaticPowerLoad formulation
└ @ PowerSimulations /Users/cbarrows/.juli

We can look at the "MD" Model

In [43]:
sim.stages["MD"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 16
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 10 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 16 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 14 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can look at the "DA" model

In [44]:
sim.stages["DA"].internal.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 144
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 121 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 144 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 144 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

And we can execute the simulation by running the following command

```julia
sim_results = execute!(sim)
```

3-Stage Simulation:

In [46]:
stages_definition = Dict(
    "MD" => Stage(GenericOpProblem, template_md, c_sys5_hy_wk, Cbc_optimizer),
    "UC" => Stage(GenericOpProblem, template_da, c_sys5_hy_uc, Cbc_optimizer),
    "ED" => Stage(GenericOpProblem, template_da, c_sys5_hy_ed, Cbc_optimizer),
)

sequence = SimulationSequence(
    order = Dict(1 => "MD", 2 => "UC", 3 => "ED"),
    intra_stage_chronologies = Dict(
        ("MD" => "UC") => Synchronize(periods = 2),
        ("UC" => "ED") => Synchronize(periods = 24),
    ),
    horizons = Dict("MD" => 2, "UC" => 24, "ED" => 12),
    intervals = Dict("MD" => Hour(48), "UC" => Hour(24), "ED" => Hour(1)),
    feed_forward = Dict(
        ("UC", :devices, :HydroDispatch) => IntegralLimitFF(
            variable_from_stage = Symbol(PSI.REAL_POWER),
            affected_variables = [Symbol(PSI.REAL_POWER)],
        ),
        ("ED", :devices, :HydroDispatch) => IntegralLimitFF(
            variable_from_stage = Symbol(PSI.REAL_POWER),
            affected_variables = [Symbol(PSI.REAL_POWER)],
        ),
    ),
    ini_cond_chronology = Dict(
        "DA" => Consecutive(),
        "UC" => Consecutive(),
        "ED" => Consecutive(),
    ),
)

SimulationSequence()


In [47]:
sim = Simulation(
    name = "hydro",
    steps = 1,
    step_resolution = Hour(48),
    stages = stages_definition,
    stages_sequence = sequence,
    simulation_folder = file_path,
    verbose = true,
)

Simulation()


In [48]:
build!(sim)

┌ Warning: Initial time not defined as an argument, it will be infered from the data.
│                Initial Simulation Time set to 2024-01-01T00:00:00
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/simulation.jl:197
┌ Info: Building Stage 2-UC
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/simulation.jl:243
┌ Info: Instantiating the JuMP model
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/psi_container.jl:16
┌ Info: Building HydroDispatch with HydroDispatchReservoirFlow formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building ThermalStandard with ThermalDispatchNoMin formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/fqiGT/src/core/operations_problem.jl:386
┌ Info: Building PowerLoad with StaticPowerLoad formulation
└ @ PowerSimulations /Users/cbarrows/.juli

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*